This notebook is for exploring amlr04-20231128 data

Specifically, we explore differences in the measured depth and CTD-calculated depth

In [ ]:
import os
import numpy as np
# import pandas as pd
import xarray as xr

from esdglider import gcp, glider, utils

deployment_name = "amlr04-20231128"
mode = "delayed"


# Standard 
bucket_name = 'amlr-gliders-deployments-dev'
deployments_path = f"/home/sam_woodman_noaa_gov/{bucket_name}"
config_path = f"/home/sam_woodman_noaa_gov/glider-lab/deployment-configs"

gcp.gcs_mount_bucket("amlr-gliders-deployments-dev", deployments_path, ro=False)
deployment_info = {
    "deploymentyaml": os.path.join(config_path, f"{deployment_name}.yml"), 
    "mode": mode, 
}
paths = glider.get_path_glider(deployment_info, deployments_path)

In [ ]:
ds_raw = xr.load_dataset(paths["tsrawpath"])
df_raw = ds_raw.to_pandas()
# display(ds_raw)

# ds_eng = xr.load_dataset(paths["tsengpath"])
# df_eng = ds_eng.to_pandas()
# display(ds_eng)

# ds_sci = xr.load_dataset(paths["tsscipath"])
# df_sci = ds_sci.to_pandas()
# display(ds_sci)
ds_raw

## Depth

In [ ]:
ds_depth = utils.check_depth(ds_raw)
ds_depth

In [ ]:
df_depth = ds_depth.to_pandas()
df_depth.sort_values(by="depth_diff_abs", ascending=False)

We can see some pretty big depth differences, inlcuding many many depth differences of ~10m at 800+m of depth. These likely indicate an issue (or at least a difference) with the glider depth sensor or CTD pressure sensor, like observed in the amlr03-20231128 deployment. 

However, upon inspection, most of these big depth differences are not actually differences between m_depth and sci_water_pressure. Instead, they're because the glider science computer appeared to error during a dive and thus needed to reboot. After talking with Tony, this was likely usually because of the glidercam. 

When the computer reboots during a dive, it's first point has all of the science values from the last measured point - these values don't reset until the second point recorded by the glider. For instance (open the below dataset in Dataviewer):

In [ ]:
ds = utils.data_var_reorder(ds_raw, ["depth", "depth_ctd"])
dt = "2023-12-25"
ds_sub = ds.sel(time=slice("2023-12-25 20:05", "2023-12-25 20:08:20"))
ds_sub

Thus, in the processing script we decided to remove all of these instances . The following bounds were determined after lots of investigation. They may not catch instance where this happened and the gldier traveled less than 6 meters. However, using a depth difference of less than 6m led to false positives, and we are confident this captures the major offenders.

In [ ]:
df_curr = df_depth[(df_depth.depth_diff > 6) & (df_depth.depth_ctd < 300)]
df_curr

In the next cell we print the offending time stamps in a way that makes it easy to copy into the amlr04-20241128-delayed processing script

In [ ]:
df_curr.index.values